In [38]:
import pandas as pd

In [39]:
import csv

In [40]:
variation_test_csv = pd.read_csv('csv_files/df_final_experiment_clients.txt')

In [41]:
variation_test_csv.head()

,client_id,Variation
0,9988021,Test
1,8320017,Test
2,4033851,Control
3,1982004,Test
4,9294070,Control


In [42]:
footprint1_csv = pd.read_csv('csv_files/df_final_web_data_pt_1.txt')

In [43]:
footprint1_csv.head()

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04


In [44]:
footprint2_csv.shape

(412264, 5)

In [45]:
footprint2_csv = pd.read_csv('csv_files/df_final_web_data_pt_2.txt')

In [46]:
footprint2_csv.head()

,client_id,visitor_id,visit_id,process_step,date_time
0,763412,601952081_10457207388,397475557_40440946728_419634,confirm,2017-06-06 08:56:00
1,6019349,442094451_91531546617,154620534_35331068705_522317,confirm,2017-06-01 11:59:27
2,6019349,442094451_91531546617,154620534_35331068705_522317,step_3,2017-06-01 11:58:48
3,6019349,442094451_91531546617,154620534_35331068705_522317,step_2,2017-06-01 11:58:08
4,6019349,442094451_91531546617,154620534_35331068705_522317,step_1,2017-06-01 11:57:58


In [47]:
customer_csv = pd.read_csv('csv_files/df_final_demo.txt')

In [48]:
customer_csv.head()

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,836976,6.0,73.0,60.5,U,2.0,45105.30,6.0,9.0
1,2304905,7.0,94.0,58.0,U,2.0,110860.30,6.0,9.0
2,1439522,5.0,64.0,32.0,U,2.0,52467.79,6.0,9.0
3,1562045,16.0,198.0,49.0,M,2.0,67454.65,3.0,6.0
4,5126305,12.0,145.0,33.0,F,2.0,103671.75,0.0,3.0


In [49]:
footprint_csv = pd.concat([footprint1_csv, footprint2_csv], ignore_index=True)

In [50]:
footprint_csv.shape

(755405, 5)

In [51]:
footprint_csv["process_step"].value_counts() 

process_step
start      243945
step_1     163193
step_2     133062
step_3     112242
confirm    102963
Name: count, dtype: int64

In [52]:
process_replace = {"start": 0, "step_1": 1, "step_2": 2, "step_3": 3, "confirm": 4}

In [53]:
footprint_csv["process_step"]= footprint_csv["process_step"].replace(process_replace)

C:\Users\greta\AppData\Local\Temp\ipykernel_46632\3095718887.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  footprint_csv["process_step"]= footprint_csv["process_step"].replace(process_replace)


In [54]:
footprint_csv["process_step"].value_counts() 

process_step
0    243945
1    163193
2    133062
3    112242
4    102963
Name: count, dtype: int64

In [55]:
variation_test_csv["Variation"].isna().sum()

20109

In [56]:
variation_test_csv["Variation"].value_counts()

Variation
Test       26968
Control    23532
Name: count, dtype: int64

In [57]:
variation_replace = {"Test": 1, "Control": 0}

In [58]:
variation_test_csv["Variation"]= variation_test_csv["Variation"].replace(variation_replace)

C:\Users\greta\AppData\Local\Temp\ipykernel_46632\1689626888.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  variation_test_csv["Variation"]= variation_test_csv["Variation"].replace(variation_replace)


In [59]:
variation_test_csv["Variation"].value_counts()

Variation
1.0    26968
0.0    23532
Name: count, dtype: int64

In [60]:
variation_test_csv["Variation"].isna().sum()

20109

In [61]:
#drop null values in variation colums
variation_test_csv = variation_test_csv.dropna(subset=['Variation'])

In [62]:
variation_test_csv["Variation"].isna().sum()

0

In [63]:
#convertir variation a int
variation_test_csv["Variation"] = variation_test_csv["Variation"].astype(int)

In [64]:
variation_test_csv["Variation"].value_counts()

Variation
1    26968
0    23532
Name: count, dtype: int64

In [66]:
customer_csv["clnt_tenure_yr"].isna().sum()

14

In [68]:
customer_csv = customer_csv.dropna(subset=['clnt_tenure_yr'])

In [69]:
customer_csv["clnt_tenure_yr"] = customer_csv["clnt_tenure_yr"].astype(int)

In [70]:
customer_csv["clnt_tenure_mnth"].isna().sum()

0

In [71]:
customer_csv["clnt_tenure_mnth"] = customer_csv["clnt_tenure_mnth"].astype(int)

In [72]:
#num_accts
customer_csv["num_accts"].isna().sum()	

0

In [73]:
customer_csv["num_accts"] = customer_csv["num_accts"].astype(int)

In [74]:
#calls_6_mnth
customer_csv["calls_6_mnth"].isna().sum()

0

In [75]:
customer_csv["calls_6_mnth"] = customer_csv["calls_6_mnth"].astype(int)

In [77]:
#logons_6_mnth
customer_csv["logons_6_mnth"].isna().sum()

0

In [78]:
customer_csv["logons_6_mnth"] = customer_csv["logons_6_mnth"].astype(int)

In [79]:
customer_csv.head()

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,836976,6,73,60.5,U,2,45105.30,6,9
1,2304905,7,94,58.0,U,2,110860.30,6,9
2,1439522,5,64,32.0,U,2,52467.79,6,9
3,1562045,16,198,49.0,M,2,67454.65,3,6
4,5126305,12,145,33.0,F,2,103671.75,0,3


In [80]:
footprint_csv.head()

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,3,2017-04-17 15:18:04


In [81]:
variation_test_csv.head()

,client_id,Variation
0,9988021,1
1,8320017,1
2,4033851,0
3,1982004,1
4,9294070,0


In [82]:
footprint_csv["process_step"].isna().sum()

0

In [84]:
pip Matplotlib installed

Note: you may need to restart the kernel to use updated packages.


ERROR: unknown command "Matplotlib"



In [85]:
import matplotlib.pyplot as plt

In [86]:
pip Seaborn installed

Note: you may need to restart the kernel to use updated packages.


ERROR: unknown command "Seaborn" - maybe you meant "search"



In [87]:
import seaborn as sns